# Station Metadata

Extracta metadata for major airport stations from the Environment Canada weather station inventory and save to CSV file.

In [1]:
import pandas as pd

In [2]:
# Airport stations we are interested in
airport_timezones = {'YEG': 'MST', 'YFB': 'EST', 'YFC': 'AST', 'YHZ': 'AST', 'YOW': 'EST',
                     'YQB': 'EST', 'YQR': 'CST', 'YQT': 'EST', 'YUL': 'EST', 'YVR': 'PST',
                     'YWG': 'CST', 'YXE': 'CST', 'YXS': 'PST', 'YXY': 'PST', 'YYC': 'MST',
                     'YYG': 'AST', 'YYJ': 'PST', 'YYT': 'NST', 'YYZ': 'EST', 'YZF': 'MST'}

stations = list(airport_timezones.keys())

In [3]:
# Timezone offsets from UTC time
utc_offset_hours = {'AST' : -4, 'CST' : -6, 'EST' : -5, 'MST' : -7, 'NST' : -3.5, 'PST' : -8}

In [4]:
# Complete station inventory from Environment Canada
inventory = pd.read_csv('data/Station Inventory EN.csv', skiprows=3)

# Rename some of the columns to more convenient labels
cols_map = {'TC ID' : 'Airport Code',
            'Station ID' : 'Env Canada ID',
            'Latitude (Decimal Degrees)' : 'Latitude (deg)',
            'Longitude (Decimal Degrees)' : 'Longitude (deg)'}
inventory = inventory.rename(columns=cols_map)

print(inventory.shape)
inventory.head()

(8737, 19)


,Name,Province,Climate ID,Env Canada ID,WMO ID,Airport Code,Latitude (deg),Longitude (deg),Latitude,Longitude,Elevation (m),First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year
0,ACTIVE PASS,BRITISH COLUMBIA,1010066,14,NaN,NaN,48.87,-123.28,485200000,-1231700000,4.0,1984,1996,NaN,NaN,1984.0,1996.0,1984.0,1996.0
1,ALBERT HEAD,BRITISH COLUMBIA,1010235,15,NaN,NaN,48.40,-123.48,482400000,-1232900000,17.0,1971,1995,NaN,NaN,1971.0,1995.0,1971.0,1995.0
2,BAMBERTON OCEAN CEMENT,BRITISH COLUMBIA,1010595,16,NaN,NaN,48.58,-123.52,483500000,-1233100000,85.3,1961,1980,NaN,NaN,1961.0,1980.0,1961.0,1980.0
3,BEAR CREEK,BRITISH COLUMBIA,1010720,17,NaN,NaN,48.50,-124.00,483000000,-1240000000,350.5,1910,1971,NaN,NaN,1910.0,1971.0,1910.0,1971.0
4,BEAVER LAKE,BRITISH COLUMBIA,1010774,18,NaN,NaN,48.50,-123.35,483000000,-1232100000,61.0,1894,1952,NaN,NaN,1894.0,1952.0,1894.0,1952.0


In [5]:
# Find rows that correspond to selected stations and which have recent data
base_year = 2018
in_station_list = inventory['Airport Code'].isin(stations)
has_data = inventory['Last Year'] >= base_year
stations_info = inventory[in_station_list & has_data]

# Sort table according to Airport Code
stations_info = stations_info.sort_values('Airport Code')

# There's two entries for Thunder Bay, so manually drop one of them
stations_info = stations_info[stations_info['Env Canada ID'] != 49389]

# Set Airport Code as the index
stations_info = stations_info.set_index('Airport Code')

# Add timezone info
stations_info['Timezone'] = stations_info.index.to_series().map(airport_timezones)
stations_info['UTC Offset (hours)'] = stations_info['Timezone'].map(utc_offset_hours)

# Keep only the columns we're interested in
cols_keep = ['Name', 'Province', 'Timezone', 'UTC Offset (hours)', 'Env Canada ID', 'Latitude (deg)', 
             'Longitude (deg)', 'Elevation (m)', 'First Year', 'Last Year']
stations_info = stations_info[cols_keep]

# Save station info to file
stations_file = 'data/airport_stations.csv'
print(f'Saving to {stations_file}')
stations_info.to_csv(stations_file)

stations_info

Saving to data/airport_stations.csv


,Name,Province,Timezone,UTC Offset (hours),Env Canada ID,Latitude (deg),Longitude (deg),Elevation (m),First Year,Last Year
Airport Code,,,,,,,,,,
YEG,EDMONTON INTL A,ALBERTA,MST,-7.0,50149,53.31,-113.58,723.3,2012,2018
YFB,IQALUIT A,NUNAVUT,EST,-5.0,52079,63.76,-68.56,33.5,2014,2018
YFC,FREDERICTON,NEW BRUNSWICK,AST,-4.0,48568,45.87,-66.54,20.7,2010,2018
YHZ,HALIFAX INTL A,NOVA SCOTIA,AST,-4.0,50620,44.88,-63.51,145.4,2012,2018
YOW,OTTAWA INTL A,ONTARIO,EST,-5.0,49568,45.32,-75.67,114.9,2011,2018
YQB,QUEBEC INTL A,QUEBEC,EST,-5.0,51457,46.79,-71.39,74.4,2013,2018
YQR,REGINA INTL A,SASKATCHEWAN,CST,-6.0,51441,50.43,-104.67,577.6,2013,2018
YQT,THUNDER BAY A,ONTARIO,EST,-5.0,50132,48.37,-89.32,199.3,2012,2018
YUL,MONTREAL INTL A,QUEBEC,EST,-5.0,51157,45.47,-73.74,36.0,2013,2018
